# Seoul Bike Sharing — EDA + Limpieza (Modified) y Comparación vs Original (Referencia) + Baseline ML

**Reglas**
- Trabajamos y modelamos con **`seoul_bike_sharing_modified.csv`** (principal).
- **`seoul_bike_sharing_original.csv`** es **solo referencia** para validar limpieza/distribución.
- Ambos CSV en el **mismo directorio** que este notebook en Drive. El notebook puede *auto-descubrir* si no das la ruta.


## 1) Setup e imports

In [ ]:
#@title Setup e imports
import os, sys, warnings, json, hashlib, textwrap, math
warnings.filterwarnings("ignore")
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
print('Versions -> numpy', np.__version__, '| pandas', pd.__version__)

## 2) Ubicación de archivos

In [ ]:
#@title Montar Drive y resolver rutas (mismo directorio del notebook)
use_drive = True #@param {type:"boolean"}
folder_path = "" #@param {type:"string"}
fname_original = "seoul_bike_sharing_original.csv"; fname_modified = "seoul_bike_sharing_modified.csv"
if use_drive:
    from google.colab import drive; drive.mount('/content/drive')
    base = Path('/content/drive/MyDrive')
    if folder_path:
        data_dir = Path(folder_path); assert data_dir.exists(), f"Ruta no encontrada: {data_dir}"
        p_org, p_mod = data_dir/fname_original, data_dir/fname_modified
    else:
        print("🔎 Buscando archivos en tu Drive...")
        orgs = list(base.rglob(fname_original)); mods = list(base.rglob(fname_modified))
        assert orgs and mods, "No se encontraron ambos CSV. Especifica 'folder_path'."
        commons = {p.parent for p in orgs} & {p.parent for p in mods}
        if commons:
            data_dir = sorted(commons)[0]; p_org, p_mod = data_dir/fname_original, data_dir/fname_modified
        else:
            print("⚠️ Están en carpetas distintas. Se usarán rutas individuales."); p_org, p_mod = orgs[0], mods[0]
else:
    from google.colab import files
    print("👉 Sube los dos CSV"); up = files.upload()
    assert 'seoul_bike_sharing_original.csv' in up and 'seoul_bike_sharing_modified.csv' in up
    p_org = Path('/content/seoul_bike_sharing_original.csv'); p_mod = Path('/content/seoul_bike_sharing_modified.csv')
print("Original:", p_org); print("Modified:", p_mod)

## 3) Carga (principal=modified, referencia=original)

In [ ]:
#@title Cargar
org = pd.read_csv(p_org); mod = pd.read_csv(p_mod)
org.head(2), mod.head(2)

In [ ]:
mod

In [ ]:
org

## 4) Utilidades

In [ ]:
#@title Funciones
def normalize_cols(df):
    df = df.copy(); df.rename(columns={c: c.strip().replace("\\xa0"," ").replace("  "," ").strip() for c in df.columns}, inplace=True); return df
def add_parsed_date(df):
    df = df.copy(); d=[c for c in df.columns if "date" in c.lower()]; df["__Date"]=pd.to_datetime(df[d[0]], errors="coerce", dayfirst=True) if d else pd.NaT
    h=[c for c in df.columns if "hour" in c.lower()]; df["__Hour"]=pd.to_numeric(df[h[0]], errors="coerce") if h else np.nan; return df
def guess_target(cols):
    for c in cols:
        if "rented" in c.lower() and "count" in c.lower(): return c
    return None
def clean_df(df, target_col):
    df=df.copy()
    for c in df.select_dtypes(include=["object"]).columns:
        df[c]=df[c].astype(str).str.strip().replace({"nan": np.nan})
    for c in df.columns:
        if c!=target_col and df[c].dtype==object:
            num=pd.to_numeric(df[c].str.replace(",","").str.replace("%",""), errors="coerce")
            if num.notna().sum()>=0.5*len(df): df[c]=num
    for c in df.select_dtypes(include=[np.number]).columns:
        q1,q99=df[c].quantile(0.01), df[c].quantile(0.99)
        if pd.notna(q1) and pd.notna(q99) and q99>q1: df[c]=df[c].clip(q1,q99)
    return df
def detect_categoricals(df, target_col):
    cats=list(df.select_dtypes(include=["object","category","bool"]).columns)
    for c in df.select_dtypes(include=[np.number]).columns:
        if df[c].nunique(dropna=True)<=20 and c!=target_col: cats.append(c)
    return sorted([c for c in set(cats) if not c.startswith("__")])


## 5) Objetivo y EDA rápida

In [ ]:
#@title 🎯 Objetivo y EDA

# Normalización de columnas y fechas
org = normalize_cols(org)
mod = normalize_cols(mod)
org = add_parsed_date(org)
mod = add_parsed_date(mod)

# Detección automática de columna objetivo
target_col = guess_target(mod.columns) or guess_target(org.columns)
if target_col is None:
    n = mod.select_dtypes(include=np.number).columns
    target_col = n[0] if len(n) > 0 else mod.columns[0]

print("🔹 Columna objetivo detectada:", target_col)

# --- Función de resumen (overview) ---
def overview(df, name):
    info = pd.DataFrame({
        "col": df.columns,
        "dtype": df.dtypes.astype(str),
        "n_missing": df.isna().sum().values,
        "pct_missing": (100 * df.isna().sum() / len(df)).round(2).values,
        "n_unique": [df[c].nunique(dropna=True) for c in df.columns]
    })
    display(info.sort_values("pct_missing", ascending=False).style.set_caption(name))

# Mostrar resúmenes de ambos conjuntos
overview(mod, "Modified (pre-limpieza)")
overview(org, "Original (referencia)")

# --- Análisis de la variable objetivo ---
plt.figure(figsize=(8, 5))

# Limpieza básica de la columna objetivo (quita $, comas, espacios)
s = mod[target_col].astype(str).str.replace(r"[,\s\$\€]", "", regex=True)
num = pd.to_numeric(s, errors="coerce")  # convierte a numérico, fuerza NaN si hay texto

print(f"Total de filas: {len(num)}")
print(f"Valores válidos: {num.notna().sum()}  |  Nulos o no convertibles: {num.isna().sum()}")

# Graficar distribución
num.dropna().astype(float).plot(kind="hist", bins=30)
plt.title(f"Distribución del objetivo (Modified, pre-limpieza): {target_col}")
plt.xlabel(target_col)
plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()


In [ ]:
#@title 🔍 Análisis forense de `mixed_type_col` (¿conviene eliminarla?)
import numpy as np, pandas as pd, re, warnings
from collections import Counter
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score

warnings.filterwarnings("ignore")

# --- Verificaciones previas ---
assert 'mod' in globals(), "No encuentro el DataFrame `mod` (ejecuta celdas 3-5)."
assert 'target_col' in globals(), "No encuentro `target_col` (ejecuta Celda 5)."

COL = 'mixed_type_col'
if COL not in mod.columns:
    raise KeyError(f"No existe `{COL}` en mod.columns")

print("=== 1) Perfil básico de la columna ===")
s = mod[COL]
print("dtype:", s.dtype)
n = len(s)
n_missing = s.isna().sum()
n_unique = s.nunique(dropna=True)
print(f"filas={n:,} | n_missing={n_missing:,} ({100*n_missing/n:.2f}%) | n_unique={n_unique:,} ({100*n_unique/n:.2f}% del total)")

types_counts = Counter(type(x).__name__ for x in s)
print("Tipos de valores observados:", dict(types_counts))

top_vals = pd.Series(s.astype(str)).value_counts(dropna=False).head(12)
print("\nTop 12 valores (crudos):")
print(top_vals)

print("\n=== 2) ¿Se puede convertir a numérico? ===")
sn = (
    s.astype(str)
     .str.strip()
     .str.replace(r"[,\s%]", "", regex=True)
     .str.replace(r"[^\-\.\dEe+]", "", regex=True)
)
num = pd.to_numeric(sn, errors='coerce')
pct_numeric = 100 * num.notna().mean()
print(f"% convertibles a número: {pct_numeric:.2f}%")
if num.notna().sum() > 0:
    print("Resumen (solo convertibles):")
    print(num.dropna().astype(float).describe())

print("\n=== 3) Asociación con el target ===")
y = pd.to_numeric(mod[target_col], errors='coerce')
mask = y.notna()
y_valid = y[mask]

if num.notna().sum() > 0:
    corr = np.corrcoef(num[mask].fillna(num[mask].median()), y_valid)[0,1]
    print(f"Correlación numérica con {target_col}: {corr:.4f}")
else:
    print("No hay suficientes valores numéricos para correlación directa.")

if s.nunique(dropna=True) <= 50:
    grp = pd.DataFrame({"col": s[mask], "y": y_valid}).dropna().groupby("col")["y"].mean().sort_values(ascending=False)
    print("\nMedia del target por categoría (top 10):")
    print(grp.head(10))

print("\n=== 4) ¿Predice demasiado bien por sí sola? ===")
df_tmp = mod[[COL, target_col]].copy()
df_tmp[target_col] = pd.to_numeric(df_tmp[target_col], errors='coerce')
df_tmp = df_tmp.dropna(subset=[target_col]).copy()
X_one = df_tmp[[COL]]
y_one = df_tmp[target_col].astype(float)

preproc = ColumnTransformer([
    ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), [COL])
])
model_one = Pipeline(steps=[
    ('enc', preproc),
    ('rf', RandomForestRegressor(n_estimators=200, random_state=42))
])

cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores_r2 = cross_val_score(model_one, X_one, y_one, scoring='r2', cv=cv)
scores_rmse = -cross_val_score(model_one, X_one, y_one, scoring='neg_root_mean_squared_error', cv=cv)

print(f"CV R² (solo `{COL}`):  mean={scores_r2.mean():.4f} ±{scores_r2.std():.4f}")
print(f"CV RMSE (solo `{COL}`): mean={scores_rmse.mean():.2f} ±{scores_rmse.std():.2f}")

print("\n=== 5) Comparación con y sin la columna ===")
mod_numish = mod.copy()
for c in mod_numish.select_dtypes('object').columns:
    coerced = pd.to_numeric(mod_numish[c].astype(str).str.replace(r"[,\s%]", "", regex=True), errors='coerce')
    if coerced.notna().mean() >= 0.6:
        mod_numish[c] = coerced

A = mod_numish.dropna(subset=[target_col]).copy()
A_y = pd.to_numeric(A[target_col], errors='coerce')
A = A.select_dtypes(include=[np.number]).drop(columns=[target_col], errors='ignore')
A = A.loc[A_y.notna()]
A_y = A_y.loc[A_y.notna()].astype(float)

X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(A, A_y, test_size=0.2, random_state=42)
rf_A = RandomForestRegressor(n_estimators=200, random_state=42)
rf_A.fit(X_train_A, y_train_A)
r2_A = r2_score(y_test_A, rf_A.predict(X_test_A))
print(f"R² (CON `{COL}`): {r2_A:.4f}")

B = mod_numish.drop(columns=[COL], errors='ignore')
B = B.dropna(subset=[target_col]).copy()
B_y = pd.to_numeric(B[target_col], errors='coerce')
B = B.select_dtypes(include=[np.number]).drop(columns=[target_col], errors='ignore')
B = B.loc[B_y.notna()]
B_y = B_y.loc[B_y.notna()].astype(float)

X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(B, B_y, test_size=0.2, random_state=42)
rf_B = RandomForestRegressor(n_estimators=200, random_state=42)
rf_B.fit(X_train_B, y_train_B)
r2_B = r2_score(y_test_B, rf_B.predict(X_test_B))
print(f"R² (SIN `{COL}`): {r2_B:.4f}")

print("\n=== 6) Señales empíricas ===")
flags = []
if n_unique / n > 0.9:
    flags.append("cardinalidad_altisima")
if len(types_counts) > 1:
    flags.append("tipos_mezclados")
if scores_r2.mean() >= 0.2:
    flags.append("predice_demasiado_bien_sola")
if (r2_A - r2_B > 0.05):
    flags.append("impacto_excesivo_en_modelo")

print("Flags activadas:", flags if flags else "ninguna")

print("\n=== 7) Recomendación preliminar ===")
if flags:
    print("→ Sugerencia: ELIMINAR `mixed_type_col` del modelado. Motivos:")
    if "cardinalidad_altisima" in flags: print("  - Cardinalidad ~única por fila (parece ID o hash).")
    if "tipos_mezclados" in flags: print("  - Mezcla de tipos (object con strings/números/NaN).")
    if "predice_demasiado_bien_sola" in flags: print("  - Predice demasiado bien por sí sola (posible fuga de información).")
    if "impacto_excesivo_en_modelo" in flags: print("  - Aporta ganancia anómala al R² y alta importancia en el modelo.")
else:
    print("→ No hay señales fuertes de fuga; mantenla con cautela y valida con CV/temporal splits.")


## 6) Limpieza SOLO en modified

In [ ]:
#@title Limpieza y guardado
mod_clean = clean_df(mod, target_col)
mod_clean[target_col] = pd.to_numeric(mod_clean[target_col], errors="coerce")

# Eliminar columna problemática si existe
if "mixed_type_col" in mod_clean.columns:
    mod_clean.drop(columns=["mixed_type_col"], inplace=True)
    print("Columna 'mixed_type_col' eliminada.")

# Guardar solo el dataset limpio modificado
out_dir = Path(p_mod).parent if 'p_mod' in globals() else Path('.')
mod_clean_path = out_dir / 'cleaned_modified.csv'
mod_clean.to_csv(mod_clean_path, index=False)

print("Guardado:", mod_clean_path)
print("Tamaño final ->", mod_clean.shape)


## 7) ¿Se conserva la distribución? Comparación vs original

In [ ]:
#@title Stats lado a lado + KS test (reconstruye org_clean al vuelo)
from scipy.stats import ks_2samp
import pandas as pd

# --- Reconstruir org_clean al vuelo (sin depender de la Celda 6) ---
if 'p_org' not in globals():
    raise RuntimeError("No encuentro 'p_org'. Ejecuta la Celda 2 (montaje/carga de rutas) antes de esta.")

# Cargar original crudo
org = pd.read_csv(p_org)

# Normalización y fechas (usa tus utilidades de la Celda 4 si existen)
if 'normalize_cols' in globals():
    org = normalize_cols(org)
if 'add_parsed_date' in globals():
    org = add_parsed_date(org)

# Limpiar con la misma función usada en mod
if 'clean_df' not in globals():
    raise RuntimeError("No encuentro 'clean_df'. Ejecuta la Celda 4 (funciones) antes de esta.")
org_clean = clean_df(org, target_col)

# Asegurar que el target es numérico (por consistencia)
org_clean[target_col] = pd.to_numeric(org_clean[target_col], errors="coerce")

# --- Comparación KS entre mod_clean (ya existente) y org_clean (reconstruido) ---
common = [c for c in mod_clean.columns if c in org_clean.columns]
rows = []

for c in common:
    a = pd.to_numeric(mod_clean[c], errors="coerce").dropna()
    b = pd.to_numeric(org_clean[c], errors="coerce").dropna()
    if len(a) > 50 and len(b) > 50:
        stat, p = ks_2samp(a, b)
        rows.append({
            "col": c,
            "mean_mod": a.mean(),
            "std_mod": a.std(),
            "p50_mod": a.median(),
            "mean_org": b.mean(),
            "std_org": b.std(),
            "p50_org": b.median(),
            "ks_stat": stat,
            "ks_pvalue": p
        })

cmp_df = pd.DataFrame(rows).sort_values("ks_stat", ascending=False)

if len(cmp_df) > 0:
    display(cmp_df.style.set_caption(
        "Comparación (modified limpio vs original limpio) — KS p>0.05 ≈ sin cambio estadístico fuerte"
    ))
else:
    print("⚠️ No hubo suficientes columnas numéricas con datos en ambos datasets para comparar con KS.")


## 8) Baseline ML (solo modified limpio)

In [ ]:
#@title 8 BIS) Baseline ML

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, classification_report
from sklearn.inspection import permutation_importance

# ------------------ Parámetros editables ------------------
TEST_SIZE = 0.15
VAL_SIZE  = 0.15
RANDOM_STATE = 42
DO_STANDARDIZE = False          # árboles no lo necesitan
N_ESTIMATORS = 500              # un poco más robusto
MAX_DEPTH = 20                  # profundidad moderada
MIN_SAMPLES_LEAF = 2            # hoja mínima para reducir overfitting
MAX_FEATURES = 'sqrt'           # estrategia común en RF
N_PERMUT = 10
TOPK_IMP = 15
USE_SEASONS_AS_ORDINAL = False  # Seasons como nominal por ciclo anual
# ----------------------------------------------------------

# --- Chequeos previos ---
if 'mod_clean' not in globals():
    raise RuntimeError("No se encontró 'mod_clean'. Ejecuta la celda de limpieza primero.")
if 'target_col' not in globals() or target_col not in mod_clean.columns:
    raise KeyError("No se encontró 'target_col' o no está en mod_clean.columns")

# Copia de trabajo + salvaguarda por si mixed sigue ahí
df = mod_clean.copy()
if "mixed_type_col" in df.columns:
    df = df.drop(columns=["mixed_type_col"])

# Target numérico + drop filas sin target
df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
df = df.dropna(subset=[target_col]).copy()

# ---------- Feature engineering cíclico para Hour ----------
# Si existe Hour (o __Hour), convierte a num y crea sin/cos
hour_candidates = [c for c in ['Hour', '__Hour'] if c in df.columns]
if hour_candidates:
    hcol = hour_candidates[0]
    df[hcol] = pd.to_numeric(df[hcol], errors='coerce')
    # reemplazar outliers/NaN razonablemente
    df[hcol] = df[hcol].clip(0, 23)
    df['Hour_sin'] = np.sin(2*np.pi*df[hcol]/24.0)
    df['Hour_cos'] = np.cos(2*np.pi*df[hcol]/24.0)
    # (Opcional) si quieres evitar doble conteo, puedes eliminar la original:
    # df.drop(columns=[hcol], inplace=True)

y = df[target_col].astype(float)
X = df.drop(columns=[target_col])

# Detectar tipos base
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_all  = [c for c in X.columns if c not in num_cols]

# --- Definir ordinales vs nominales ---
ordinal_specs_all = []

# Seasons -> por defecto NOMINAL (USE_SEASONS_AS_ORDINAL=False)
if 'Seasons' in X.columns and USE_SEASONS_AS_ORDINAL:
    ordinal_specs_all.append(("Seasons", ["Winter", "Spring", "Summer", "Autumn"]))

# Functioning Day: binaria ordenada (si existe)
if 'Functioning Day' in X.columns:
    ordinal_specs_all.append(("Functioning Day", ["No", "Yes"]))

ordinal_cols   = [name for (name, order) in ordinal_specs_all if name in X.columns]
ordinal_orders = [order for (name, order) in ordinal_specs_all if name in X.columns]
cat_nominal = [c for c in cat_all if c not in ordinal_cols]

print(f"Dataset final: {df.shape} | Numéricas: {len(num_cols)} | "
      f"Cat nominales: {len(cat_nominal)} | Cat ordinales: {len(ordinal_cols)} | Target n: {len(y)}")

# ---------- Estratificación (binning si regresión) ----------
is_classification = False
if y.nunique() <= 20:
    is_classification = True

if is_classification:
    stratify_labels = y
else:
    n_bins = min(10, int(np.sqrt(len(y))))
    try:
        stratify_labels = pd.qcut(y, q=n_bins, duplicates='drop')
    except Exception:
        stratify_labels = pd.cut(y, bins=n_bins)

# ---------- Split: test y luego val ----------
X_rest, X_test, y_rest, y_test, strat_rest, strat_test = train_test_split(
    X, y, stratify_labels, test_size=TEST_SIZE, random_state=RANDOM_STATE
)
val_frac_of_rest = VAL_SIZE / (1.0 - TEST_SIZE)
X_train, X_val, y_train, y_val = train_test_split(
    X_rest, y_rest, test_size=val_frac_of_rest, random_state=RANDOM_STATE, stratify=strat_rest
)

print(f"Split sizes -> train: {len(X_train)}, val: {len(X_val)}, test: {len(X_test)}")

# ---------- Preprocesamiento ----------
num_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler() if DO_STANDARDIZE else "passthrough")
])

# OneHot compatible con distintas versiones de sklearn
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)  # sklearn >= 1.2
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)         # sklearn < 1.2

cat_nominal_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", ohe)
])

cat_ordinal_transformer = None
if ordinal_cols:
    cat_ordinal_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ordinal", OrdinalEncoder(
            categories=ordinal_orders,
            handle_unknown="use_encoded_value",
            unknown_value=-1
        ))
    ])

transformers = []
if num_cols:
    transformers.append(("num", num_transformer, num_cols))
if cat_nominal:
    transformers.append(("cat_nom", cat_nominal_transformer, cat_nominal))
if ordinal_cols and cat_ordinal_transformer is not None:
    transformers.append(("cat_ord", cat_ordinal_transformer, ordinal_cols))

preproc = ColumnTransformer(
    transformers=transformers,
    remainder="drop",
    verbose_feature_names_out=False
)

# ---------- Modelo ----------
if is_classification:
    model = RandomForestClassifier(
        n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, random_state=RANDOM_STATE,
        n_jobs=-1
    )
else:
    model = RandomForestRegressor(
        n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, random_state=RANDOM_STATE,
        min_samples_leaf=MIN_SAMPLES_LEAF, max_features=MAX_FEATURES, n_jobs=-1
    )

pipe = Pipeline(steps=[("preproc", preproc), ("model", model)])

# Entrenar
pipe.fit(X_train, y_train)

# Predecir
y_val_pred = pipe.predict(X_val)
y_test_pred = pipe.predict(X_test)

# ---------- Métricas ----------
if is_classification:
    print("\nVALIDACIÓN - Clasificación:")
    print(classification_report(y_val, y_val_pred))
    print("TEST - Clasificación:")
    print(classification_report(y_test, y_test_pred))
    print("Test accuracy:", accuracy_score(y_test, y_test_pred))
else:
    print("\nVALIDACIÓN - Regresión:")
    print("MAE:", mean_absolute_error(y_val, y_val_pred))
    print("MSE:", mean_squared_error(y_val, y_val_pred))
    print("RMSE:", np.sqrt(mean_squared_error(y_val, y_val_pred)))
    print("R2:", r2_score(y_val, y_val_pred))
    print("\nTEST - Regresión (evaluación final):")
    print("MAE:", mean_absolute_error(y_test, y_test_pred))
    print("MSE:", mean_squared_error(y_test, y_test_pred))
    print("RMSE:", np.sqrt(mean_squared_error(y_test, y_test_pred)))
    print("R2:", r2_score(y_test, y_test_pred))

# ---------- Importancia por permutación (alineada a columnas de ENTRADA) ----------
try:
    input_feature_names = []
    if num_cols:      input_feature_names += list(num_cols)
    if cat_nominal:   input_feature_names += list(cat_nominal)
    if ordinal_cols:  input_feature_names += list(ordinal_cols)

    result = permutation_importance(
        pipe, X_val, y_val, n_repeats=N_PERMUT, random_state=RANDOM_STATE, n_jobs=-1
    )
    imp = pd.Series(result.importances_mean, index=input_feature_names).sort_values(ascending=False)
    print(f"\nTop {min(TOPK_IMP, len(imp))} features por permutación (nivel columnas de entrada):")
    display(imp.head(TOPK_IMP))
except Exception as e:
    print("No se pudo calcular la importancia por permutación:", e)

print("\nNotas:")
print("- Seasons se trata como NOMINAL por defecto (switch USE_SEASONS_AS_ORDINAL=False).")
print("- Hour ahora incluye Hour_sin/Hour_cos; se puede eliminar la columna Hour original si se dejo.")
print("- Árboles sin escalado; RF con min_samples_leaf y max_features para mejor generalización.")
print("- Guarda el pipeline con joblib.dump(pipe, 'best_model.joblib').")


In [ ]:
#@title 9) XGBoost + features temporales/derivadas + LOG1P + MAPE + permutación

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.inspection import permutation_importance

# ------------------ Parámetros editables ------------------
TEST_SIZE = 0.15
VAL_SIZE  = 0.15
RANDOM_STATE = 42

# Preprocesamiento
DO_STANDARDIZE_NUM = False      # boosting no lo necesita
USE_SEASONS_AS_ORDINAL = False  # Seasons como nominal (ciclo anual)
USE_HOUR_SIN_COS = True         # mantener Hour_sin / Hour_cos si se generaron antes

# Modelo (ajusta aquí)
TRY_XGBOOST = True              # si no está instalado, fallback a HistGradientBoosting
XGB_PARAMS = dict(
    n_estimators=700,
    max_depth=6,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=RANDOM_STATE,
    tree_method="hist",
    n_jobs=-1
)
HGB_PARAMS = dict(
    max_depth=None, learning_rate=0.05, max_iter=600, random_state=RANDOM_STATE
)

# Target
LOG1P_TARGET = True            # activar para estabilizar varianza del target
N_PERMUT = 10
TOPK_IMP = 15
# ----------------------------------------------------------

# --- Chequeos y base ---
if 'mod_clean' not in globals():
    raise RuntimeError("No se encontró 'mod_clean'. Ejecuta la celda de limpieza primero.")
if 'target_col' not in globals() or target_col not in mod_clean.columns:
    raise KeyError("No se encontró 'target_col' o no está en mod_clean.columns")

df = mod_clean.copy()
if "mixed_type_col" in df.columns:
    df = df.drop(columns=["mixed_type_col"])

# --- Features temporales desde Date ---
def ensure_datetime(s):
    return pd.to_datetime(s, errors='coerce')

if 'Date' in df.columns:
    dt = ensure_datetime(df['Date'])
    df['Year']    = dt.dt.year
    df['Month']   = dt.dt.month
    df['Weekday'] = dt.dt.dayofweek  # 0=Lunes ... 6=Domingo
    df['is_weekend'] = (df['Weekday'] >= 5).astype(int)
    # Codificación cíclica Month/Weekday
    df['Month_sin']   = np.sin(2*np.pi*(df['Month']-1)/12.0)
    df['Month_cos']   = np.cos(2*np.pi*(df['Month']-1)/12.0)
    df['Weekday_sin'] = np.sin(2*np.pi*df['Weekday']/7.0)
    df['Weekday_cos'] = np.cos(2*np.pi*df['Weekday']/7.0)

# Si existen Hour/__Hour y no se crearon sin/cos antes, activarlo aquí
if USE_HOUR_SIN_COS:
    for hcol in ['Hour', '__Hour']:
        if hcol in df.columns:
            df[hcol] = pd.to_numeric(df[hcol], errors='coerce').clip(0, 23)
            if 'Hour_sin' not in df.columns:
                df['Hour_sin'] = np.sin(2*np.pi*df[hcol]/24.0)
            if 'Hour_cos' not in df.columns:
                df['Hour_cos'] = np.cos(2*np.pi*df[hcol]/24.0)
            # (Opcional) evita duplicidad si dejas sin/cos
            # df.drop(columns=[hcol], inplace=True)
            break

# --- Features DERIVADAS útiles ---
# Hora pico
if 'Hour' in df.columns:
    df['is_rush_hour'] = df['Hour'].isin([7,8,9,17,18,19]).astype(int)
elif '__Hour' in df.columns:
    df['is_rush_hour'] = df['__Hour'].isin([7,8,9,17,18,19]).astype(int)

# Comodidad percibida aprox.
if {'Temperature(°C)', 'Humidity(%)'}.issubset(df.columns):
    df['comfort_index'] = df['Temperature(°C)'] - df['Humidity(%)']/5.0

# Disconfort por viento
if {'Wind speed (m/s)', 'Humidity(%)'}.issubset(df.columns):
    df['wind_discomfort'] = df['Wind speed (m/s)'] * df['Humidity(%)']

# Feriado o fin de semana (normaliza Holiday a binaria)
if 'is_weekend' in df.columns and 'Holiday' in df.columns:
    h = df['Holiday'].astype(str).str.lower().isin(['holiday','yes','1','true'])
    df['is_holiday_or_weekend'] = ((df['is_weekend'] == 1) | h).astype(int)

# --- Target y split base ---
df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
df = df.dropna(subset=[target_col]).copy()

if LOG1P_TARGET:
    y_raw = df[target_col].astype(float)
    y = np.log1p(y_raw)
else:
    y = df[target_col].astype(float)

X = df.drop(columns=[target_col])

# Detectar tipos
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_all  = [c for c in X.columns if c not in num_cols]

# Definir ordinales vs nominales
ordinal_specs_all = []
if 'Seasons' in X.columns and USE_SEASONS_AS_ORDINAL:
    ordinal_specs_all.append(("Seasons", ["Winter", "Spring", "Summer", "Autumn"]))
if 'Functioning Day' in X.columns:
    ordinal_specs_all.append(("Functioning Day", ["No", "Yes"]))

ordinal_cols   = [name for (name, order) in ordinal_specs_all if name in X.columns]
ordinal_orders = [order for (name, order) in ordinal_specs_all if name in X.columns]
cat_nominal = [c for c in cat_all if c not in ordinal_cols]

print(f"Dataset final: {df.shape} | Numéricas: {len(num_cols)} | "
      f"Cat nominales: {len(cat_nominal)} | Cat ordinales: {len(ordinal_cols)} | Target n: {len(y)}")

# Estratificación por bins (regresión)
n_bins = min(10, int(np.sqrt(len(y))))
try:
    stratify_labels = pd.qcut(y, q=n_bins, duplicates='drop')
except Exception:
    stratify_labels = pd.cut(y, bins=n_bins)

X_rest, X_test, y_rest, y_test, strat_rest, strat_test = train_test_split(
    X, y, stratify_labels, test_size=TEST_SIZE, random_state=RANDOM_STATE
)
val_frac_of_rest = VAL_SIZE / (1.0 - TEST_SIZE)
X_train, X_val, y_train, y_val = train_test_split(
    X_rest, y_rest, test_size=val_frac_of_rest, random_state=RANDOM_STATE, stratify=strat_rest
)
print(f"Split sizes -> train: {len(X_train)}, val: {len(X_val)}, test: {len(X_test)}")

# --- Preprocesamiento ---
num_steps = [("imputer", SimpleImputer(strategy="median"))]
if DO_STANDARDIZE_NUM:
    num_steps.append(("scaler", StandardScaler()))
num_transformer = Pipeline(steps=num_steps)

# OneHot compatible versiones
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)  # sklearn >= 1.2
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)         # sklearn < 1.2

cat_nominal_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", ohe)
])

cat_ordinal_transformer = None
if ordinal_cols:
    cat_ordinal_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ordinal", OrdinalEncoder(
            categories=ordinal_orders,
            handle_unknown="use_encoded_value",
            unknown_value=-1
        ))
    ])

transformers = []
if num_cols:
    transformers.append(("num", num_transformer, num_cols))
if cat_nominal:
    transformers.append(("cat_nom", cat_nominal_transformer, cat_nominal))
if ordinal_cols and cat_ordinal_transformer is not None:
    transformers.append(("cat_ord", cat_ordinal_transformer, ordinal_cols))

preproc = ColumnTransformer(
    transformers=transformers,
    remainder="drop",
    verbose_feature_names_out=False
)

# --- Modelo: XGBoost o fallback a HistGradientBoosting ---
model = None
used_xgb = False
if TRY_XGBOOST:
    try:
        from xgboost import XGBRegressor
        model = XGBRegressor(**XGB_PARAMS)
        used_xgb = True
        print("Modelo: XGBRegressor")
    except Exception as e:
        print("XGBoost no disponible o falló la importación; usando HistGradientBoostingRegressor.", e)

if model is None:
    from sklearn.ensemble import HistGradientBoostingRegressor
    model = HistGradientBoostingRegressor(**HGB_PARAMS)
    print("Modelo: HistGradientBoostingRegressor")

pipe = Pipeline(steps=[("preproc", preproc), ("model", model)])

# ---------- (Opcional) Búsqueda automática rápida ----------
# from sklearn.model_selection import RandomizedSearchCV
# param_grid = {
#     "model__n_estimators": [400, 600, 800],
#     "model__learning_rate": [0.03, 0.05, 0.1],
#     "model__max_depth": [4, 6, 8],
#     "model__subsample": [0.7, 0.8, 0.9],
#     "model__colsample_bytree": [0.7, 0.8, 1.0],
#     "model__reg_lambda": [0.5, 1.0, 2.0]
# }
# search = RandomizedSearchCV(pipe, param_distributions=param_grid, n_iter=12,
#                             scoring="r2", cv=3, n_jobs=-1, random_state=42, verbose=1)
# search.fit(X_train, y_train)
# pipe = search.best_estimator_
# print("Mejores parámetros:", search.best_params_)

# Entrenar
pipe.fit(X_train, y_train)

# Predecir
y_val_pred = pipe.predict(X_val)
y_test_pred = pipe.predict(X_test)

# Invertir transformación del target para métricas en escala original
def inv_target(z):
    return np.expm1(z) if LOG1P_TARGET else z

y_val_eval = inv_target(y_val)
y_test_eval = inv_target(y_test)
y_val_pred_eval = inv_target(y_val_pred)
y_test_pred_eval = inv_target(y_test_pred)

# Métricas + MAPE
def report_split(name, y_true, y_hat):
    mae = mean_absolute_error(y_true, y_hat)
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_hat)

    # MAPE evitando división por cero
    m = (np.abs((y_true - y_hat)[y_true != 0] / y_true[y_true != 0])).mean() * 100 if np.any(y_true != 0) else np.nan

    print(f"{name}:")
    print(f"  MAE:  {mae:.3f}")
    print(f"  MSE:  {mse:.3f}")
    print(f"  RMSE: {rmse:.3f}")
    print(f"  R2:   {r2:.6f}")
    print(f"  MAPE: {m:.2f}%")

print("\nVALIDACIÓN - Regresión:")
report_split("VAL", y_val_eval, y_val_pred_eval)

print("\nTEST - Regresión (evaluación final):")
report_split("TEST", y_test_eval, y_test_pred_eval)

# --- Importancia por permutación (nivel columnas de entrada) ---
try:
    input_feature_names = []
    if num_cols:      input_feature_names += list(num_cols)
    if cat_nominal:   input_feature_names += list(cat_nominal)
    if ordinal_cols:  input_feature_names += list(ordinal_cols)

    result = permutation_importance(
        pipe, X_val, y_val, n_repeats=N_PERMUT, random_state=RANDOM_STATE, n_jobs=-1
    )
    imp = pd.Series(result.importances_mean, index=input_feature_names).sort_values(ascending=False)
    print(f"\nTop {min(TOPK_IMP, len(imp))} features por permutación (entrada):")
    display(imp.head(TOPK_IMP))
except Exception as e:
    print("No se pudo calcular la importancia por permutación:", e)

print("\nNotas:")
print("- Añadidas: is_rush_hour, comfort_index, wind_discomfort, is_holiday_or_weekend.")
print(f"- Modelo usado: {'XGBRegressor' if used_xgb else 'HistGradientBoostingRegressor'}; LOG1P_TARGET={LOG1P_TARGET}.")
print("- Ajusta XGB_PARAMS o activa la búsqueda automática (RandomizedSearchCV) para subir R².")


In [ ]:
#@title 10) Diagnóstico visual mejorado (ticks limpios, sanitización, zoom y bins)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# =========================
# Build DataFrame de evaluación
# =========================
df_eval = pd.DataFrame({"y_real": y_test_eval, "y_pred": y_test_pred_eval})
df_eval["residual"] = df_eval["y_real"] - df_eval["y_pred"]

if "X_test" in locals():
    df_plot = pd.concat([df_eval.reset_index(drop=True), X_test.reset_index(drop=True)], axis=1)
else:
    df_plot = df_eval.copy()

# =========================
# Helpers
# =========================
def yclip(series, q=0.99):
    """Devuelve un (min, max) simétrico por cuantil absoluto para 'zoom'."""
    lim = series.abs().quantile(q)
    lim = float(lim) if np.isfinite(lim) and lim > 0 else float(series.abs().max() or 1.0)
    return (-lim, lim)

def safe_num(s, lo=None, hi=None, round_to_int=False, allow_na_int=True, clip=True):
    """Convierte a numérico, limpia inf, recorta y opcionalmente redondea/castea a entero con NA."""
    x = pd.to_numeric(s, errors="coerce").replace([np.inf, -np.inf], np.nan)
    if clip and (lo is not None or hi is not None):
        x = x.clip(lower=lo, upper=hi)
    if round_to_int:
        x = x.round()
        if allow_na_int:
            x = x.astype("Int64")  # entero que acepta NA
        else:
            x = x.fillna(0).astype(int)
    return x

# =========================
# 1) Pred vs Real
# =========================
plt.figure(figsize=(7,6))
sns.scatterplot(data=df_plot, x="y_real", y="y_pred", s=15, alpha=0.6)
minv = float(df_plot[["y_real","y_pred"]].min().min())
maxv = float(df_plot[["y_real","y_pred"]].max().max())
plt.plot([minv, maxv], [minv, maxv], ls="--", color="red", label="y = x")
plt.legend(loc="upper left")
plt.title("Predicción vs Real (Test)")
plt.xlabel("Real"); plt.ylabel("Predicho")
plt.tight_layout(); plt.show()

# =========================
# 2) Distribución de residuales (completa y con zoom)
# =========================
plt.figure(figsize=(8,4))
sns.histplot(df_plot["residual"], bins=60, kde=True)
plt.title("Distribución de residuales (y_real - y_pred)")
plt.xlabel("Residual")
plt.tight_layout(); plt.show()

low, high = yclip(df_plot["residual"], 0.995)
plt.figure(figsize=(8,4))
sns.histplot(df_plot["residual"].clip(low, high), bins=60, kde=True)
plt.title("Distribución de residuales (zoom 99.5%)")
plt.xlabel("Residual (recortado)")
plt.tight_layout(); plt.show()

# =========================
# 3) Residuales vs Predicción (zoom 99%)
# =========================
plt.figure(figsize=(7,4))
ylim = yclip(df_plot["residual"], 0.99)
sns.scatterplot(data=df_plot, x="y_pred", y=df_plot["residual"].clip(*ylim), s=10, alpha=0.6)
plt.axhline(0, ls="--", color="red")
plt.ylim(ylim)
plt.title("Residuales vs Predicción (zoom 99%)")
plt.xlabel("Predicción"); plt.ylabel("Residual")
plt.tight_layout(); plt.show()

# =========================
# 4) Por HORA: boxplot + MAE (con sanitización)
# =========================
if "Hour" in df_plot.columns or "__Hour" in df_plot.columns:
    hcol = "Hour" if "Hour" in df_plot.columns else "__Hour"
    tmp = df_plot.copy()
    # Sanitizar a [0,23], entero con NA permitido
    tmp[hcol] = safe_num(tmp[hcol], lo=0, hi=23, round_to_int=True, allow_na_int=True, clip=True)
    tmp["Hour_str"] = tmp[hcol].astype(str).fillna("NA")

    ylim = yclip(tmp["residual"], 0.99)
    plt.figure(figsize=(12,4))
    sns.boxplot(data=tmp, x="Hour_str", y=tmp["residual"].clip(*ylim))
    plt.ylim(ylim); plt.title("Residuales por Hour (zoom 99%)")
    plt.xlabel("Hour"); plt.ylabel("Residual")
    plt.tight_layout(); plt.show()

    # MAE por hora (excluye NA)
    mae_hour = (
        tmp[tmp["Hour_str"] != "NA"]
        .groupby(hcol)["residual"]
        .apply(lambda s: s.abs().mean())
        .sort_index()
    )
    plt.figure(figsize=(12,3))
    mae_hour.plot(kind="bar")
    plt.title("MAE por Hour")
    plt.xlabel("Hour"); plt.ylabel("MAE")
    plt.tight_layout(); plt.show()

# =========================
# 5) Por WEEKDAY: boxplot + MAE (con sanitización)
# =========================
if "Weekday" in df_plot.columns:
    tmp = df_plot.copy()
    tmp["Weekday"] = safe_num(tmp["Weekday"], lo=0, hi=6, round_to_int=True, allow_na_int=True, clip=True)
    day_labels = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
    # Mapear con NA seguro
    mapper = dict(enumerate(day_labels))
    tmp["Weekday_lbl"] = tmp["Weekday"].map(mapper).astype(object)
    tmp["Weekday_lbl"] = tmp["Weekday_lbl"].fillna("NA")

    ylim = yclip(tmp["residual"], 0.99)
    plt.figure(figsize=(9,4))
    sns.boxplot(data=tmp, x="Weekday_lbl", y=tmp["residual"].clip(*ylim))
    plt.ylim(ylim); plt.title("Residuales por Weekday (zoom 99%)")
    plt.xlabel("Weekday"); plt.ylabel("Residual")
    plt.tight_layout(); plt.show()

    # MAE por weekday (excluye NA)
    mae_wd = (
        tmp[tmp["Weekday_lbl"] != "NA"]
        .groupby("Weekday_lbl")["residual"]
        .apply(lambda s: s.abs().mean())
        .reindex(day_labels)  # asegurar orden Lun..Dom
    )
    plt.figure(figsize=(9,3))
    mae_wd.plot(kind="bar")
    plt.title("MAE por Weekday")
    plt.xlabel("Weekday"); plt.ylabel("MAE")
    plt.tight_layout(); plt.show()

# =========================
# 6) Temperatura / Humedad: agrupar en BINS (qcut) y MAE por bin
# =========================
def boxplot_by_bins(df_in, col, bins=10, label=None):
    if col not in df_in.columns:
        return
    tmp = df_in[[col, "residual"]].dropna().copy()
    if tmp.empty:
        return
    # Bins por cuantiles; si hay duplicados de borde, qcut los maneja
    try:
        tmp["bin"] = pd.qcut(tmp[col], q=bins, duplicates="drop")
    except ValueError:
        # Si hay muy pocos valores distintos
        tmp["bin"] = pd.cut(tmp[col], bins=min(bins, 5))
    ylim = yclip(tmp["residual"], 0.99)

    plt.figure(figsize=(12,4))
    sns.boxplot(data=tmp, x="bin", y=tmp["residual"].clip(*ylim))
    plt.xticks(rotation=45, ha="right")
    plt.ylim(ylim); plt.title(f"Residuales por {label or col} (bins, zoom 99%)")
    plt.xlabel(label or col); plt.ylabel("Residual")
    plt.tight_layout(); plt.show()

    mae_bin = tmp.groupby("bin")["residual"].apply(lambda s: s.abs().mean())
    plt.figure(figsize=(12,3))
    mae_bin.plot(kind="bar")
    plt.xticks(rotation=45, ha="right")
    plt.title(f"MAE por {label or col} (bins)")
    plt.xlabel(label or col); plt.ylabel("MAE")
    plt.tight_layout(); plt.show()

boxplot_by_bins(df_plot, "Temperature(°C)", bins=12, label="Temperature(°C)")
boxplot_by_bins(df_plot, "Humidity(%)",     bins=12, label="Humidity(%)")

# =========================
# 7) Correlaciones de residuales con numéricas
# =========================
num_cols_eval = df_plot.select_dtypes(include=[np.number]).columns
if "residual" in num_cols_eval:
    corr_resid = df_plot[num_cols_eval].corr()["residual"].sort_values(ascending=False)
    print("🔹 Correlaciones de residuales con variables numéricas (positivas = sobreestimación):")
    display(corr_resid.head(15))

# =========================
# 8) Top outliers para inspección
# =========================
topk = 10
print(f"\n🔎 Top {topk} residuales absolutos (para inspección):")
cols_show = ["y_real","y_pred","residual"]
for c in ["Date","Hour","__Hour","Weekday","Temperature(°C)","Humidity(%)","Rainfall(mm)","Snowfall (cm)","Holiday","is_weekend","is_holiday_or_weekend"]:
    if c in df_plot.columns:
        cols_show.append(c)

top_df = (
    df_plot.reindex(columns=[c for c in cols_show if c in df_plot.columns])
    .iloc[np.argsort(-df_plot['residual'].abs())[:topk]]
)
display(top_df)
